In [1]:
%load_ext autoreload
%autoreload 2

# MLflow HuggingFace Recipe Notebook

This notebook runs the MLflow HuggingFace Recipe and inspects its results. For more information about the MLflow HuggingFace Recipe, including usage examples, see the [HuggingFace Recipe overview documentation](https://mlflow.org/docs/latest/recipes.html#huggingface-recipe) the [HuggingFace Recipe API documentation](https://mlflow.org/docs/latest/python_api/mlflow.recipes.html#module-mlflow.recipes.huggingface.v1.recipe).

In [2]:
from mlflow.recipes import Recipe

r = Recipe(profile="local")

2023/03/05 21:40:32 INFO mlflow.recipes.recipe: Creating MLflow Recipe 'huggingface' with profile: 'local'


In [3]:
r.clean()

In [4]:
r.inspect()

In [5]:
r.run("ingest")

2023/03/05 21:40:34 INFO mlflow.recipes.step: Running step ingest...
Using custom data configuration default-b37f4d81be795426
Found cached dataset text (/Users/jin.zhang/.cache/huggingface/datasets/text/default-b37f4d81be795426/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)

100%|██████████| 1/1 [00:00<00:00, 926.71it/s]

Creating parquet from Arrow format: 100%|██████████| 33/33 [00:00<00:00, 4657.67ba/s]


In [6]:
r.run("split")

2023/03/05 21:40:35 INFO mlflow.recipes.utils.execution: ingest: No changes. Skipping.


Run MLFlow Recipe step: split
2023/03/05 21:40:36 INFO mlflow.recipes.step: Running step split...


In [7]:
r.run("transform")

2023/03/05 21:40:39 INFO mlflow.recipes.utils.execution: ingest, split: No changes. Skipping.


Run MLFlow Recipe step: transform
2023/03/05 21:40:40 INFO mlflow.recipes.step: Running step transform...


In [8]:
r.run("train")

2023/03/05 21:40:46 INFO mlflow.recipes.utils.execution: ingest, split, transform: No changes. Skipping.


Run MLFlow Recipe step: train
2023/03/05 21:40:47 INFO mlflow.recipes.step: Running step train...
2023/03/05 21:40:47 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/03/05 21:40:47 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [datasets.builder] Using custom data configuration default-53aeda4e5941a5c6


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 1045.57it/s]

Generating train split: 0 examples [00:00, ? examples/s]
                                                        

Generating validation split: 0 examples [00:00, ? examples/s]
                                                             
Dataset parquet downloaded and prepared to /Users/jin.zhang/.cache/huggingface/datasets

2023/03/05 21:41:01 INFO mlflow.recipes.step: Unable to locate runtime info for step 'train'. Re-run the step before inspect.


In [10]:
r.run("evaluate")

2023/03/05 21:43:52 INFO mlflow.recipes.utils.execution: ingest, split, transform: No changes. Skipping.


Run MLFlow Recipe step: train
2023/03/05 21:43:53 INFO mlflow.recipes.step: Running step train...
2023/03/05 21:43:54 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/03/05 21:43:54 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [datasets.builder] Using custom data configuration default-53aeda4e5941a5c6
WARNI [datasets.builder] Found cached dataset parquet (/Users/jin.zhang/.cache/huggingface/datasets/parquet/default-53aeda4e5941a5c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)

100%|██████████| 2/2 [00:00<00:00, 841.38it/s]
WARNI [azureml.core] Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:Script

2023/03/05 21:44:39 INFO mlflow.recipes.step: Unable to locate runtime info for step 'evaluate'. Re-run the step before inspect.


In [11]:
r.run("register")

2023/03/05 21:45:08 INFO mlflow.recipes.utils.execution: ingest, split, transform: No changes. Skipping.


Run MLFlow Recipe step: train
2023/03/05 21:45:09 INFO mlflow.recipes.step: Running step train...
2023/03/05 21:45:10 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/03/05 21:45:10 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [datasets.builder] Using custom data configuration default-53aeda4e5941a5c6
WARNI [datasets.builder] Found cached dataset parquet (/Users/jin.zhang/.cache/huggingface/datasets/parquet/default-53aeda4e5941a5c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)

100%|██████████| 2/2 [00:00<00:00, 899.20it/s]
WARNI [azureml.core] Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:Script

In [12]:
r.inspect("train")

2023/03/05 21:46:22 INFO mlflow.recipes.step: Unable to locate runtime info for step 'train'. Re-run the step before inspect.


In [13]:
run = r.get_artifact("run")
print(run)

2023/03/05 21:46:27 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/03/05 21:46:27 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Run: data=<RunData: metrics={}, params={}, tags={'mlflow.gitRepoURL': 'git@github.com:mlflow/recipes-examples.git',
 'mlflow.pipeline.profile.name': 'local',
 'mlflow.pipeline.step.name': 'register',
 'mlflow.pipeline.template.name': 'huggingface/v1',
 'mlflow.runName': 'casual-turtle-465',
 'mlflow.source.git.branch': 'huggingface',
 'mlflow.source.git.commit': 'fca6289767a7504187ca8d86e6f7e283180e485f',
 'mlflow.source.git.repoURL': 'git@github.com:mlflow/recipes-examples.git',
 'mlflow.source.name': 'git@github.com:mlflow/recipes-examples.git',
 'mlflow.source.type': 'RECIPE',
 'mlflow.user': 'jin.zhang'}>, info=<RunInfo: artifact_uri='/Users/jin.zhang/Code/recipes-examples/huggingface/metadata/mlflow/mlartifacts/18cbcadd983e4d57a9f83554d2341014/artifacts', end_time=1678081529754, experiment_id='1', lifecycle_stage='active', run_id='18cbcadd983e4d57a9f83554d2341014', run_name='casual-turtle-465', run_uuid='18cbcadd983e4d57a9f83554d2341014', start_time=1678081510675, status='FINISHE

In [14]:
model = r.get_artifact("model")
print(model)

OSError: No such file or directory: '/Users/jin.zhang/Code/recipes-examples/huggingface/metadata/mlflow/mlartifacts/18cbcadd983e4d57a9f83554d2341014/artifacts/train/model'

In [ ]:
model.predict("Tell me what happens next")

In [ ]:
dir(model)